In [8]:
import numpy as np
import pandas as pd
import feather
#import lightgbm as lgb
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold

In [9]:
def constrain_to_range(x,upper_lim,lower_lim):
    return min(max(x, lower_lim),upper_lim)

In [14]:
#def create_lgbm_function(static_parameters, optimize_vars):

def lgbm_evaluate(num_leaves, colsample_by_tree, subsample, max_depth, reg_alpha, reg_lambda, min_split_gain, min_child_weight):
    
    train = pd.read_feather("./../Solution attempts/v07 train data.feather")
    target = pd.read_feather("./../Solution attempts/v07 target.feather")
    
    params = {"bagging_fraction":0.5}
    params["n_estimators"] = 4000#int(n_estimators)
    params["learning_rate"] = 0.05
    params["num_leaves"] = int(num_leaves)
    params["colsample_by_tree"] = constrain_to_range(colsample_by_tree,0,1)
    params["subsample"] = constrain_to_range(subsample,0,1)
    params["max_depth"] = int(max_depth)
    params["reg_alpha"] = max(reg_alpha,0)
    params["reg_lambda"] = max(reg_lambda,0)
    params["min_split_gain"] = min_split_gain
    params["min_child_weight"] = min_child_weight
    
    folds = KFold(n_splits = 6, shuffle = True)
    auc_scores = []
    
    for _, (train_ids, val_ids) in enumerate(folds.split(train)):
        train_train = train.iloc[train_ids]
        train_test = train.iloc[val_ids]
        target_train = np.ravel(target.iloc[train_ids][["TARGET"]])
        target_test = np.ravel(target.iloc[val_ids][["TARGET"]])
        
        clf = LGBMClassifier(**params)
        clf.fit(train_train, target_train, eval_set = [(train_train, target_train), (train_test, target_test)],
                eval_metric = "auc", early_stopping_rounds = 50, verbose = 0)
        
        auc_scores.append(clf._best_score["valid_1"]["auc"])
    
    return(np.mean(auc_scores))

In [7]:
x = 54
func_as_string = f"""def test_func(y):
    return y + {x}"""
?eval

In [15]:
lgbm_parameter_ranges = {"num_leaves":(20,30),
                         "colsample_by_tree":(0.6,0.9),
                         "subsample":(0.7,0.95),
                         "max_depth":(4,10),
                         "reg_alpha":(0.01,0.2),
                         "reg_lambda":(0.01,0.2),
                         "min_split_gain":(0.1,0.1),
                         "min_child_weight":(2,2)}

xgb_BO = BayesianOptimization(lgbm_evaluate, lgbm_parameter_ranges)
xgb_BO.maximize(init_points = 10, n_iter = 25)

Initialization
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_by_tree |   max_depth |   min_child_weight |   min_split_gain |   num_leaves |   reg_alpha |   reg_lambda |   subsample | 
    1 | 01m57s |    0.77235 |              0.7469 |      9.4141 |             2.0000 |           0.1000 |      29.4355 |      0.1703 |       0.0542 |      0.7742 | 
    2 | 02m03s |    0.77349 |              0.6054 |      6.9346 |             2.0000 |           0.1000 |      21.8605 |      0.0696 |       0.0646 |      0.7361 | 
    3 | 01m45s |    0.77226 |              0.8019 |      9.7759 |             2.0000 |           0.1000 |      22.2292 |      0.0302 |       0.1586 |      0.7928 | 
    4 | 02m24s |    0.77327 |              0.8277 |      4.5847 |             2.0000 |           0.1000 |      27.1941 |      0.0634 |       0.0768 |      0.7819

/home/greg/.local/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([1.56590481e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 56, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/greg/.local/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_by_tree |   max_depth |   min_child_weight |   min_split_gain |   num_leaves |   reg_alpha |   reg_lambda |   subsample | 
    6 | 02m34s |    0.77394 |              0.6146 |      4.0967 |             2.0000 |           0.1000 |      20.5337 |      0.1940 |       0.1862 |      0.9265 | 


/home/greg/.local/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


    7 | 02m27s |    0.77332 |              0.6122 |      4.1087 |             2.0000 |           0.1000 |      20.1297 |      0.0415 |       0.0332 |      0.9439 | 


...

/home/greg/.local/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00023324]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 57, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/greg/.local/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   24 | 05m36s |    0.77229 |              0.8046 |      9.9386 |             2.0000 |           0.1000 |      20.0514 |      0.0176 |       0.0167 |      0.7515 | 


/home/greg/.local/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   25 | 03m49s |    0.77403 |              0.8071 |      4.0014 |             2.0000 |           0.1000 |      29.9446 |      0.1536 |       0.0190 |      0.9481 | 


/home/greg/.local/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [18]:
vars(xgb_BO)

{'_acqkw': {'n_iter': 250, 'n_warmup': 100000},
 'gp': GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
              kernel=Matern(length_scale=1, nu=2.5),
              n_restarts_optimizer=25, normalize_y=False,
              optimizer='fmin_l_bfgs_b',
              random_state=<mtrand.RandomState object at 0x7efd56b100d8>),
 'i': 20,
 'init_points': [array([29.43546615,  0.74687803,  0.77416114,  9.41414366,  0.17034076,
          0.05420865,  0.1       ,  2.        ]),
  array([21.86049638,  0.6054212 ,  0.73610302,  6.93456483,  0.06964915,
          0.06458417,  0.1       ,  2.        ]),
  array([22.2292204 ,  0.80194088,  0.79282062,  9.77594514,  0.03018998,
          0.15859723,  0.1       ,  2.        ]),
  array([27.19413811,  0.82771565,  0.78192196,  4.58468582,  0.06338625,
          0.0768028 ,  0.1       ,  2.        ]),
  array([28.38799525,  0.79220296,  0.76334472,  6.13168599,  0.05064306,
          0.09787208,  0.1       ,  2.        ])],
 'initialized':